In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset_builder
snli_dataset = load_dataset_builder("snli")

In [ ]:
# Inspect dataset description
snli_dataset.info.description

'The SNLI corpus (version 1.0) is a collection of 570k human-written English\nsentence pairs manually labeled for balanced classification with the labels\nentailment, contradiction, and neutral, supporting the task of natural language\ninference (NLI), also known as recognizing textual entailment (RTE).\n'

In [ ]:
# Inspect dataset features
snli_dataset.info.features

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [ ]:
from datasets import get_dataset_split_names
get_dataset_split_names("snli")

['test', 'train', 'validation']

In [ ]:
from datasets.load import load_dataset
snli_dataset_train = load_dataset("snli",split="train")
snli_dataset_validation = load_dataset("snli",split="validation")

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


In [ ]:
snli_dataset_train[1]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is at a diner, ordering an omelette.',
 'label': 2}

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.5 MB/s eta 0:00:00


In [ ]:
! pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.6 MB/s eta 0:00:00


In [ ]:
import torch
import random
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define function to process a single text
def process_text(text):
    premise_text = "translate English to French: "+text["premise"]
    premise_text_french = tokenizer(premise_text,padding="longest",max_length=512,truncation=True,return_tensors="pt").input_ids.to(device)
    outputs_french = model.generate(premise_text_french)
    decoded_french = tokenizer.decode(outputs_french[0], skip_special_tokens=True)

    hypothesis_text = "translate English to French: "+text["hypothesis"]
    hypothesis_text_french = tokenizer(hypothesis_text,padding="longest",max_length=512,truncation=True, return_tensors="pt").input_ids.to(device)
    outputs_hyp_french= model.generate(hypothesis_text_french)
    decoded_hyp_french = tokenizer.decode(outputs_hyp_french[0], skip_special_tokens=True)

    label = text["label"]
    return decoded_hyp_french, decoded_french, label

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Parallelize processing of texts using DataParallel module
decoded_hyp_list = []
decoded_french_list = []
labels_list = []

texts_list = snli_dataset_train
if torch.cuda.device_count() > 1:
  print("Using", torch.cuda.device_count(), "GPUs for parallel processing")
  model = torch.nn.DataParallel(model)

randomList = random.sample(range(0, len(texts_list)),57000)

print("starting the process for training data")
for i in randomList:
  decoded_hyp_french, decoded_french, label = process_text(texts_list[i])
  decoded_hyp_list.append(decoded_hyp_french)
  decoded_french_list.append(decoded_french)
  labels_list.append(label)

In [ ]:
# for validation data
decoded_hyp_list_v = []
decoded_french_list_v= []
labels_list_v = []
print("starting the process for validation")
texts_list_v = snli_dataset_validation
for j in range(0, len(texts_list_v)):
  decoded_hyp_french_v, decoded_french_v, label_v = process_text(texts_list_v[j])
  decoded_hyp_list_v.append(decoded_hyp_french_v)
  decoded_french_list_v.append(decoded_french_v)
  labels_list_v.append(label_v)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#saving the dataset in french to train csv
import csv
french_list = {"decoded_french": decoded_french_list, "labels": labels_list,"decoded_hyp_list":decoded_hyp_list}

# Open a CSV file in write mode
with open('/content/drive/My Drive/nlp244/assignments/quest-4/french_train-57000.csv', 'w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(french_list.keys())

    # Write the data rows
    writer.writerows(zip(*french_list.values()))

In [ ]:
#saving the dataset in french to validation csv
import csv
french_list = {"decoded_french": decoded_french_list_v, "labels": labels_list_v,"decoded_hyp_list":decoded_hyp_list_v}

# Open a CSV file in write mode
with open('/content/drive/My Drive/nlp244/assignments/quest-4/french_valdation.csv', 'w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(french_list.keys())

    # Write the data rows
    writer.writerows(zip(*french_list.values()))